# 1. Установка и запуск PySpark в Google Colab

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recov

In [ ]:
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Homework") \
       .getOrCreate()

spark

# Часть 1
### 1. Создайте rdd в формате ключ значение.

In [ ]:
rdd = spark.sparkContext.parallelize([("apple", 2), ("banana", 3), ("apple", 4), ("cherry", 1)])

### 2. Посчитайте среднее по ключу reduceByKey.

In [ ]:
mean_rdd = (
    rdd
    .mapValues(lambda x: (x, 1))
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
    .mapValues(lambda x: x[0] / x[1])
)
print(mean_rdd.collect())

[('banana', 3.0), ('cherry', 1.0), ('apple', 3.0)]


### 3. Посчитайте среднее по ключу через aggregateByKey.

In [ ]:
mean_rdd = (
    rdd
    .aggregateByKey((0, 0), lambda acc, value: (acc[0] + value, acc[1] + 1), lambda a, b: (a[0] + b[0], a[1] + b[1]))
    .mapValues(lambda x: x[0] / x[1])
)
print(mean_rdd.collect())

[('banana', 3.0), ('cherry', 1.0), ('apple', 3.0)]


### 4. Посчитайте среднее по ключу через combineByKey.

In [ ]:
mean_rdd = (
    rdd
    .combineByKey(
        lambda value: (value, 1),
        lambda acc, value: (acc[0] + value, acc[1] + 1),
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
    )
    .mapValues(lambda x: x[0] / x[1])
)
print(mean_rdd.collect())

[('banana', 3.0), ('cherry', 1.0), ('apple', 3.0)]


### 5. Подумайте, как можно реализовать distinct с помощью существующих трансформаций.

In [ ]:
distinct_rdd = rdd.reduceByKey(lambda a, b: a)
print(distinct_rdd.collect())

[('banana', 3), ('cherry', 1), ('apple', 2)]


# Часть 2 (с использованием RDD)
### 1. Скачайте файл

In [ ]:
import requests
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

path = "https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/shakespeare/all-shakespeare.txt"
req = requests.get(path)
url_content = req.content.decode("utf-8").splitlines()

def remove_punctuation(text):
    punctuation = r""".,"!?:;\'"`"""
    return ''.join(char.lower() for char in text if char not in punctuation).replace('\t', ' ')

url_content = [remove_punctuation(line) for line in url_content]

text_rdd = sc.parallelize(url_content)
print(text_rdd.take(10))

[' 1 king henry iv', '', '', ' dramatis personae', '', '', 'king henry the fourth (king henry iv)', '', '', 'henry']


### 2. Решите задачу wordcount для скаченного набора текста. Результат сохраните в текстовый в файл.

In [ ]:
word_counts = (
    text_rdd
    .flatMap(lambda line: line.split())
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)
output_dir = "wordcount_result"
word_counts.saveAsTextFile(output_dir)

In [ ]:
file = open('/content/wordcount_result/part-00000', mode='r')
lines = file.readlines()

for line in lines[:10]:
    print(line.strip())

file.close()

('the', 30052)
('and', 27512)
('i', 21411)
('to', 20726)
('of', 18758)
('a', 15950)
('you', 13972)
('my', 12953)
('in', 11779)
('that', 11513)


### 3. Решите задачу wordcount без стопслов (их можно либо скачать отдельно, либо использовать из библиотеки nltk) c применением broadcast. Результат сохраните в файл (parquet).

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

word_counts = (
    text_rdd
    .flatMap(lambda line: [word.lower() for word in line.split() if word.lower() not in stop_words])
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)

output_dir = "wordcount__stopwords_result"
word_counts.saveAsTextFile(output_dir)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
file = open('/content/wordcount__stopwords_result/part-00000', mode='r')
lines = file.readlines()

for line in lines[:10]:
    print(line.strip())

file.close()

('thou', 5776)
('thy', 4272)
('shall', 3721)
('thee', 3371)
('king', 3287)
('lord', 3279)
('sir', 3004)
('good', 2908)
('well', 2610)
('come', 2594)


### 4. На основе задачи 3 постройте статистику частотности встречаемости слов, н/п 10 раз встречались слова [school, love и тд], формат вывода: число – список слов через ','

In [ ]:
from collections import Counter

word_counts = (
    text_rdd
    .flatMap(lambda line: [word.lower() for word in line.split() if word.lower() not in stop_words])
    .map(lambda word: (word, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
    .collect()
)

words_by_count = {count: [word] for word, count in word_counts[:10]}
for count, words in words_by_count.items():
    if len(words) > 1:
        words_by_count[count] = sorted(words)
    print(f"{count} - {words}")

5776 - ['thou']
4272 - ['thy']
3721 - ['shall']
3371 - ['thee']
3287 - ['king']
3279 - ['lord']
3004 - ['sir']
2908 - ['good']
2610 - ['well']
2594 - ['come']


# Часть 3 (с использованием DataFrame/Spark SQL)
### 1. Решите задачу wordcount для скаченного набора текста. Результат сохраните в текстовый в файл.

In [ ]:
from pyspark.sql.functions import col, count, desc, split, lower, when, explode
df = spark.createDataFrame(url_content, StringType()).toDF("value")
wordcount_df = (df
                .select(split(lower(col("value")), " ").alias("words"))
                .withColumn("word", explode("words"))
                .groupBy("word")
                .agg(count("word").alias("num"))
                .orderBy(desc("num"))
                )

with open("wordcount_df.txt", "w") as f:
    f.write(str([(row.word, row.num) for row in wordcount_df.collect()]))

In [ ]:
wordcount_df.show(10)

+----+------+
|word|   num|
+----+------+
|    |151275|
| the| 30052|
| and| 27512|
|   i| 21411|
|  to| 20726|
|  of| 18758|
|   a| 15950|
| you| 13972|
|  my| 12953|
|  in| 11779|
+----+------+
only showing top 10 rows



### 2. Решите задачу wordcount без стопслов (их можно либо скачать отдельно, либо использовать из библиотеки nltk) c применением broadcast. Результат сохраните в файл (parquet).


In [ ]:
from pyspark.sql.functions import col, count, desc, split, lower, when, explode
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

df = spark.createDataFrame(url_content, StringType()).toDF("value")
wordcount_df = (df
                .select(split(lower(col("value")), " ").alias("words"))
                .withColumn("word", explode("words"))
                .filter(~col("word").isin(stop_words))
                .groupBy("word")
                .agg(count("word").alias("num"))
                .orderBy(desc("num"))
                )

wordcount_df.write.parquet("wordcount_df.parquet")

In [ ]:
import pandas as pd

df = pd.read_parquet('/content/wordcount_df.parquet')
print(df)

                   word     num
0                        151275
1                  thou    5776
2                   thy    4272
3                 shall    3721
4                  thee    3371
...                 ...     ...
32364              hewd       1
32365        saddle-bow       1
32366         servilely       1
32367         cupbearer       1
32368  behind-door-work       1

[32369 rows x 2 columns]


### 3. На основе задачи 3 постройте статистику частотности встречаемости слов, н/п 10 раз встречались слова [school, love и тд], формат вывода: число – список слов через ','

In [ ]:
from pyspark.sql.functions import collect_list, sort_array, size

words_by_count = (
    wordcount_df
    .groupBy('num')
    .agg(
        sort_array(collect_list('word')).alias('words')
    )
    .withColumn('word_count', size('words'))
    .sort('num', ascending=False)
)

words_by_count = {count: [word] for word, count in word_counts[:10]}
for count, words in words_by_count.items():
    if len(words) > 1:
        words_by_count[count] = sorted(words)
    print(f"{count} - {words}")

5776 - ['thou']
4272 - ['thy']
3721 - ['shall']
3371 - ['thee']
3287 - ['king']
3279 - ['lord']
3004 - ['sir']
2908 - ['good']
2610 - ['well']
2594 - ['come']
